In [59]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from source.tserie import TSerie
# from source.keras_utils import getPeaxFeatures
# from source.torch_utils import getContrastiveFeatures
# import os
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [69]:
from source.datasets import loadFuncionalModel, loadNatops, loadWafer, loadSelfRegulationSCP2, loadBasicMotions, loadEarthquakes, loadItalyPowerDemand, loadEigenWorms

dataset = 'wafer' # natops | wafer | selfreg | basicmotions | earthquakes | italy | eigenworms

if dataset == 'natops':
    X_train, y_train, X_test, y_test, classLabels = loadNatops()
elif dataset == 'wafer':
    X_train, y_train, X_test, y_test = loadWafer()
    classLabels = {}
elif dataset == 'basicmotions':
    X_train, y_train, X_test, y_test, classLabels= loadBasicMotions()
elif dataset == 'italy':
    X_train, y_train, X_test, y_test, classLabels= loadItalyPowerDemand()


In [70]:
mts_train = TSerie(X=X_train.transpose([0, 2, 1]), y=y_train)
mts_test = TSerie(X=X_test.transpose([0, 2, 1]), y=y_test)

Loaded mts - N: 1000, T: 152, D: 1 
Loaded mts - N: 6164, T: 152, D: 1 


In [62]:
# mts.smooth(window_size=40)
# mts.shapeNormalizization()
# mts.minMaxNormalizization()

In [63]:
mts_train.folding_features_v2()
mts_test.folding_features_v2()

Features shape: (40, 600)
Features shape: (40, 600)


In [64]:
import umap
import matplotlib.pyplot as plt

# reducer = umap.UMAP(n_components=2, metric='cosine')
reducer = umap.UMAP(n_components=32, metric='euclidean')
mts_train.features = reducer.fit_transform(mts_train.features)
mts_test.features = reducer.transform(mts_test.features)


In [65]:
from sklearn import svm
from sklearn.metrics import f1_score

clf = svm.SVC()
clf.fit(mts_train.features, mts_train.y)

print('accuracy')
print(clf.score(mts_train.features, mts_train.y))
print(clf.score(mts_test.features, mts_test.y))

print('f1 score')
print(f1_score(clf.predict(mts_train.features), mts_train.y, average= 'weighted'))
print(f1_score(clf.predict(mts_test.features), mts_test.y, average= 'weighted'))

accuracy
0.5
0.275
f1 score
0.5822510822510822
0.4025046382189239


In [66]:
import matplotlib.pyplot as plt
import umap

# reducer = umap.UMAP(n_components=2, metric='cosine')
reducer = umap.UMAP(n_components=2, metric='euclidean', n_neighbors=15)
reducer.fit(mts_train.features, y=mts_train.y)
coords_train = reducer.transform(mts_train.features)
coords_test = reducer.transform(mts_test.features)

# plt.scatter(coords_train[:, 0], coords_train[:, 1])


In [67]:
import sys
sys.path.append('/home/texs/Documentos/Repositories/mts_viz')
from server.source.storage import MTSStorage

In [68]:
storage = MTSStorage('mts_comparison')    
# storage.delete()
storage.load()

# classLabels = {int(v): str(v) for v in np.unique(y)}
dimensions = [str(dim) for dim in range(mts_train.D)]
# dimensions = [sPoll, sPoll + '_norm']
# dimensions = pollutants + [poll + '_norm' for poll in pollutants]
data = np.concatenate([mts_train.X_orig, mts_train.X], axis=2) 
storage.add_mts(
    '{}_umap3d_train'.format(dataset),
    data, 
    dimensions = dimensions + [dim+ '_norm' for dim in dimensions], 
    labels = {'class': mts_train.y},
    labelsNames = {'class': classLabels},
    coords = {
        'shape': coords_train,
    },
    sampling=True,
    n_samples= 400
)

data = np.concatenate([mts_test.X_orig, mts_test.X], axis=2) 
storage.add_mts(
    '{}_umap3d_test'.format(dataset),
    data, 
    dimensions = dimensions + [dim+ '_norm' for dim in dimensions], 
    labels = {'class': mts_test.y},
    labelsNames = {'class': classLabels},
    coords = {
        'shape': coords_test,
    },
    sampling=True,
    n_samples= 400
)


storage.save()

mts shape: N: 40 -  T: 100 - D: 12
mts shape: N: 40 -  T: 100 - D: 12
